In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, r2_score

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [2]:
#df = pd.read_csv('../data/full_data_interaction_05_025.csv')
#df = pd.read_csv('../data/full_data_interaction_01_075.csv')
df = pd.read_csv('../data/full_data_interaction_025_075.csv')

In [6]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
random_test_X = random_test.drop(columns=['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
random_test_y = random_test['unemployment_rate_2010']

In [9]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)
random_test_Z = ss.transform(random_test_X)

### Linear Regression on original data

In [10]:
df_lin_reg = pd.read_csv('../data/final_work_data.csv')

In [11]:
df_lin_reg.head()

,state,county_name,unemployment_rate_2010,population_total_2010,urban_population_prc,rural_population_prc,crime_per_capita,per_capita_sme_num,per_capita_large_num,avg_ann_pay_per_emp_sme,...,popul_college_grad_prc,popul_single_paren_prc,liquor_stores_per10k,police_per_1000,WhiteNonHispanicPct2010,BlackNonHispanicPct2010,AsianNonHispanicPct2010,NativeAmericanNonHispanicPct2010,HispanicPct2010,MultipleRacePct2010
0,AL,Autauga,8.8,54571.0,0.579978,0.420022,0.002620,0.012259,0.002565,22667.03037,...,0.21,0.09,0.4,1.172784,77.25,17.58,0.86,0.40,2.40,1.59
1,AL,Baldwin,9.9,182265.0,0.577209,0.422791,0.002112,0.020108,0.002019,28664.84962,...,0.26,0.09,0.8,1.486846,83.50,9.31,0.74,0.63,4.38,1.49
2,AL,Barbour,12.1,27457.0,0.322104,0.677896,0.000911,0.015042,0.002586,25720.31403,...,0.11,0.14,1.1,3.177696,46.75,46.69,0.39,0.22,5.05,0.94
3,AL,Bibb,11.2,22915.0,0.316474,0.683526,0.002575,0.011172,0.000916,27318.29733,...,0.11,0.05,0.5,0.523674,75.02,21.92,0.10,0.28,1.77,0.89
4,AL,Blount,9.7,57322.0,0.100485,0.899515,0.000419,0.010485,0.001082,26050.59382,...,0.12,0.07,0.0,0.854820,88.89,1.26,0.20,0.50,8.07,1.19


In [12]:
lin_reg = LinearRegression()
X = df_lin_reg.drop(columns=['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df_lin_reg['unemployment_rate_2010']
X_train_lr, X_test_lr, y_train_lr, y_test_lr = train_test_split(X, y, random_state=42)
lin_reg.fit(X_train_lr, y_train_lr)

LinearRegression()

In [13]:
lin_reg.score(X_train_lr, y_train_lr),lin_reg.score(X_test_lr, y_test_lr), cross_val_score(lin_reg, X_train_lr, y_train_lr).mean() 

(0.4299423542202645, 0.39382629086847676, 0.3593874243714773)

### Linear Regression on preporcessed data

In [14]:
lr = LinearRegression()
lr.fit(Z_train, y_train)

LinearRegression()

In [15]:
lr.score(Z_train, y_train), lr.score(Z_test, y_test), cross_val_score(lr, Z_train, y_train).mean()

(0.544725150185152, 0.4836054146441289, 0.45721636211613725)

In [16]:
lin_train = cross_val_predict(lr, Z_train, y_train)
lin_test = cross_val_predict(lr, Z_test, y_test)

In [17]:
lr.predict(random_test_Z), random_test_y

(array([13.41836939]),
 8    14.6
 Name: unemployment_rate_2010, dtype: object)

### Linear Regression PCA

In [18]:
df = pd.read_csv('../data/final_work_data.csv')

In [19]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [21]:
pipe = Pipeline([
    ('poly', PolynomialFeatures(degree=3, include_bias=False)),
    ('ss', StandardScaler()),
    ('pca', PCA()),
    ('model', LinearRegression())
])
params = {
    'pca__n_components': range(10,200,20)
}
gs = GridSearchCV(pipe, param_grid=params)
gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.score(X_test, y_test))
print(cross_val_score(gs, X_train, y_train).mean())
gs.best_params_

0.09325947056401707
0.2108993857426148
-4207.404376029028


{'pca__n_components': 10}

### Neural network on linear regression initeraction data

In [22]:
df = pd.read_csv('../data/full_data_interaction_025_075.csv')

In [23]:
X = df.drop(columns = ['state', 'county_name', 'unemployment_rate_2010', 'population_total_2010'])
y = df['unemployment_rate_2010']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [25]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [26]:
in_shape = Z_train[0].shape
in_shape

(97,)

In [27]:
model_1 = Sequential()

model_1.add(Dense(128, activation= 'relu', input_shape= in_shape))
model_1.add(Dropout(.3))

model_1.add(BatchNormalization())
model_1.add(Dense(128, activation= 'relu'))

model_1.add(BatchNormalization())
model_1.add(Dense(64, activation= 'relu'))

model_1.add(Dense(32, activation= 'relu'))
model_1.add(Dropout(.5))

model_1.add(Dense(1, activation= None))

In [28]:
model_1.compile(optimizer= 'adam', loss= 'mse', metrics= ['mse', 'RootMeanSquaredError', 'MeanAbsolutePercentageError'])

In [29]:
history_1 = model_1.fit(Z_train, y_train,
          epochs= 300,
          validation_data= (Z_test, y_test),
          verbose= 1
         )

Epoch 1/300
71/71 [==============================] - 1s 6ms/step - loss: 39.8278 - mse: 39.8278 - root_mean_squared_error: 6.3109 - mean_absolute_percentage_error: 55.6852 - val_loss: 29.2767 - val_mse: 29.2767 - val_root_mean_squared_error: 5.4108 - val_mean_absolute_percentage_error: 48.9044
Epoch 2/300
71/71 [==============================] - 0s 3ms/step - loss: 14.9877 - mse: 14.9877 - root_mean_squared_error: 3.8714 - mean_absolute_percentage_error: 34.3548 - val_loss: 16.7880 - val_mse: 16.7880 - val_root_mean_squared_error: 4.0973 - val_mean_absolute_percentage_error: 34.4327
Epoch 3/300
71/71 [==============================] - 0s 3ms/step - loss: 13.9982 - mse: 13.9982 - root_mean_squared_error: 3.7414 - mean_absolute_percentage_error: 33.2859 - val_loss: 10.6116 - val_mse: 10.6116 - val_root_mean_squared_error: 3.2575 - val_mean_absolute_percentage_error: 25.2471
Epoch 4/300
71/71 [==============================] - 0s 3ms/step - loss: 13.7000 - mse: 13.7000 - root_mean_squared

Epoch 29/300
71/71 [==============================] - 0s 5ms/step - loss: 9.1187 - mse: 9.1187 - root_mean_squared_error: 3.0197 - mean_absolute_percentage_error: 25.4539 - val_loss: 4.2058 - val_mse: 4.2058 - val_root_mean_squared_error: 2.0508 - val_mean_absolute_percentage_error: 16.6416
Epoch 30/300
71/71 [==============================] - 0s 4ms/step - loss: 8.9345 - mse: 8.9345 - root_mean_squared_error: 2.9891 - mean_absolute_percentage_error: 25.5416 - val_loss: 4.2795 - val_mse: 4.2795 - val_root_mean_squared_error: 2.0687 - val_mean_absolute_percentage_error: 16.5916
Epoch 31/300
71/71 [==============================] - 0s 3ms/step - loss: 9.1591 - mse: 9.1591 - root_mean_squared_error: 3.0264 - mean_absolute_percentage_error: 25.8262 - val_loss: 4.3619 - val_mse: 4.3619 - val_root_mean_squared_error: 2.0885 - val_mean_absolute_percentage_error: 16.4424
Epoch 32/300
71/71 [==============================] - 0s 5ms/step - loss: 8.7441 - mse: 8.7441 - root_mean_squared_error: 2.

71/71 [==============================] - 0s 5ms/step - loss: 6.9586 - mse: 6.9586 - root_mean_squared_error: 2.6379 - mean_absolute_percentage_error: 22.3525 - val_loss: 4.1369 - val_mse: 4.1369 - val_root_mean_squared_error: 2.0339 - val_mean_absolute_percentage_error: 16.0812
Epoch 58/300
71/71 [==============================] - 0s 3ms/step - loss: 6.8321 - mse: 6.8321 - root_mean_squared_error: 2.6138 - mean_absolute_percentage_error: 21.8751 - val_loss: 3.9143 - val_mse: 3.9143 - val_root_mean_squared_error: 1.9784 - val_mean_absolute_percentage_error: 16.3047
Epoch 59/300
71/71 [==============================] - 0s 4ms/step - loss: 6.4173 - mse: 6.4173 - root_mean_squared_error: 2.5332 - mean_absolute_percentage_error: 21.6211 - val_loss: 3.8559 - val_mse: 3.8559 - val_root_mean_squared_error: 1.9636 - val_mean_absolute_percentage_error: 16.0849
Epoch 60/300
71/71 [==============================] - 0s 4ms/step - loss: 6.8414 - mse: 6.8414 - root_mean_squared_error: 2.6156 - mean_a

71/71 [==============================] - 0s 4ms/step - loss: 5.4665 - mse: 5.4665 - root_mean_squared_error: 2.3381 - mean_absolute_percentage_error: 19.7015 - val_loss: 3.9290 - val_mse: 3.9290 - val_root_mean_squared_error: 1.9822 - val_mean_absolute_percentage_error: 15.8325
Epoch 86/300
71/71 [==============================] - 0s 4ms/step - loss: 5.4315 - mse: 5.4315 - root_mean_squared_error: 2.3306 - mean_absolute_percentage_error: 19.8925 - val_loss: 3.7299 - val_mse: 3.7299 - val_root_mean_squared_error: 1.9313 - val_mean_absolute_percentage_error: 15.6396
Epoch 87/300
71/71 [==============================] - 0s 4ms/step - loss: 5.8091 - mse: 5.8091 - root_mean_squared_error: 2.4102 - mean_absolute_percentage_error: 20.0711 - val_loss: 3.6621 - val_mse: 3.6621 - val_root_mean_squared_error: 1.9137 - val_mean_absolute_percentage_error: 15.9984
Epoch 88/300
71/71 [==============================] - 0s 5ms/step - loss: 5.5382 - mse: 5.5382 - root_mean_squared_error: 2.3533 - mean_a

71/71 [==============================] - 0s 1ms/step - loss: 4.6219 - mse: 4.6219 - root_mean_squared_error: 2.1499 - mean_absolute_percentage_error: 17.8563 - val_loss: 3.8385 - val_mse: 3.8385 - val_root_mean_squared_error: 1.9592 - val_mean_absolute_percentage_error: 15.6547
Epoch 114/300
71/71 [==============================] - 0s 1ms/step - loss: 4.7901 - mse: 4.7901 - root_mean_squared_error: 2.1886 - mean_absolute_percentage_error: 17.7995 - val_loss: 3.7967 - val_mse: 3.7967 - val_root_mean_squared_error: 1.9485 - val_mean_absolute_percentage_error: 15.6618
Epoch 115/300
71/71 [==============================] - 0s 2ms/step - loss: 4.6942 - mse: 4.6942 - root_mean_squared_error: 2.1666 - mean_absolute_percentage_error: 18.1335 - val_loss: 3.6841 - val_mse: 3.6841 - val_root_mean_squared_error: 1.9194 - val_mean_absolute_percentage_error: 15.7318
Epoch 116/300
71/71 [==============================] - 0s 2ms/step - loss: 4.6174 - mse: 4.6174 - root_mean_squared_error: 2.1488 - mea

...

71/71 [==============================] - 0s 3ms/step - loss: 3.1376 - mse: 3.1376 - root_mean_squared_error: 1.7713 - mean_absolute_percentage_error: 14.9512 - val_loss: 3.5566 - val_mse: 3.5566 - val_root_mean_squared_error: 1.8859 - val_mean_absolute_percentage_error: 15.7761
Epoch 170/300
71/71 [==============================] - 0s 1ms/step - loss: 3.1004 - mse: 3.1004 - root_mean_squared_error: 1.7608 - mean_absolute_percentage_error: 14.9923 - val_loss: 3.5722 - val_mse: 3.5722 - val_root_mean_squared_error: 1.8900 - val_mean_absolute_percentage_error: 15.6563
Epoch 171/300
71/71 [==============================] - 0s 2ms/step - loss: 3.1087 - mse: 3.1087 - root_mean_squared_error: 1.7631 - mean_absolute_percentage_error: 14.8511 - val_loss: 3.4303 - val_mse: 3.4303 - val_root_mean_squared_error: 1.8521 - val_mean_absolute_percentage_error: 15.6818
Epoch 172/300
71/71 [==============================] - 0s 1ms/step - loss: 2.9736 - mse: 2.9736 - root_mean_squared_error: 1.7244 - mea

71/71 [==============================] - 0s 2ms/step - loss: 2.7234 - mse: 2.7234 - root_mean_squared_error: 1.6503 - mean_absolute_percentage_error: 14.3624 - val_loss: 3.4264 - val_mse: 3.4264 - val_root_mean_squared_error: 1.8511 - val_mean_absolute_percentage_error: 15.5619
Epoch 198/300
71/71 [==============================] - 0s 3ms/step - loss: 2.7534 - mse: 2.7534 - root_mean_squared_error: 1.6593 - mean_absolute_percentage_error: 14.1738 - val_loss: 3.4825 - val_mse: 3.4825 - val_root_mean_squared_error: 1.8662 - val_mean_absolute_percentage_error: 15.4586
Epoch 199/300
71/71 [==============================] - 0s 2ms/step - loss: 2.7409 - mse: 2.7409 - root_mean_squared_error: 1.6556 - mean_absolute_percentage_error: 14.2617 - val_loss: 3.4759 - val_mse: 3.4759 - val_root_mean_squared_error: 1.8644 - val_mean_absolute_percentage_error: 15.7880
Epoch 200/300
71/71 [==============================] - 0s 2ms/step - loss: 2.7275 - mse: 2.7275 - root_mean_squared_error: 1.6515 - mea

71/71 [==============================] - 0s 2ms/step - loss: 2.4199 - mse: 2.4199 - root_mean_squared_error: 1.5556 - mean_absolute_percentage_error: 13.7835 - val_loss: 3.6483 - val_mse: 3.6483 - val_root_mean_squared_error: 1.9101 - val_mean_absolute_percentage_error: 15.7131
Epoch 226/300
71/71 [==============================] - 0s 3ms/step - loss: 2.5662 - mse: 2.5662 - root_mean_squared_error: 1.6019 - mean_absolute_percentage_error: 14.0958 - val_loss: 3.7293 - val_mse: 3.7293 - val_root_mean_squared_error: 1.9311 - val_mean_absolute_percentage_error: 16.1109
Epoch 227/300
71/71 [==============================] - 0s 2ms/step - loss: 2.5118 - mse: 2.5118 - root_mean_squared_error: 1.5849 - mean_absolute_percentage_error: 13.8095 - val_loss: 3.3760 - val_mse: 3.3760 - val_root_mean_squared_error: 1.8374 - val_mean_absolute_percentage_error: 15.3888
Epoch 228/300
71/71 [==============================] - 0s 3ms/step - loss: 2.4017 - mse: 2.4017 - root_mean_squared_error: 1.5497 - mea

71/71 [==============================] - 0s 2ms/step - loss: 2.3334 - mse: 2.3334 - root_mean_squared_error: 1.5275 - mean_absolute_percentage_error: 13.7921 - val_loss: 3.4401 - val_mse: 3.4401 - val_root_mean_squared_error: 1.8548 - val_mean_absolute_percentage_error: 15.3109
Epoch 254/300
71/71 [==============================] - 0s 2ms/step - loss: 2.4452 - mse: 2.4452 - root_mean_squared_error: 1.5637 - mean_absolute_percentage_error: 14.2688 - val_loss: 3.4458 - val_mse: 3.4458 - val_root_mean_squared_error: 1.8563 - val_mean_absolute_percentage_error: 15.7730
Epoch 255/300
71/71 [==============================] - 0s 3ms/step - loss: 2.3773 - mse: 2.3773 - root_mean_squared_error: 1.5418 - mean_absolute_percentage_error: 13.8873 - val_loss: 3.5389 - val_mse: 3.5389 - val_root_mean_squared_error: 1.8812 - val_mean_absolute_percentage_error: 15.4673
Epoch 256/300
71/71 [==============================] - 0s 2ms/step - loss: 2.4384 - mse: 2.4384 - root_mean_squared_error: 1.5615 - mea

71/71 [==============================] - 0s 1ms/step - loss: 2.1515 - mse: 2.1515 - root_mean_squared_error: 1.4668 - mean_absolute_percentage_error: 13.0823 - val_loss: 3.2690 - val_mse: 3.2690 - val_root_mean_squared_error: 1.8080 - val_mean_absolute_percentage_error: 15.8062
Epoch 282/300
71/71 [==============================] - 0s 2ms/step - loss: 2.0994 - mse: 2.0994 - root_mean_squared_error: 1.4489 - mean_absolute_percentage_error: 13.1735 - val_loss: 3.4688 - val_mse: 3.4688 - val_root_mean_squared_error: 1.8625 - val_mean_absolute_percentage_error: 15.6133
Epoch 283/300
71/71 [==============================] - 0s 1ms/step - loss: 2.1080 - mse: 2.1080 - root_mean_squared_error: 1.4519 - mean_absolute_percentage_error: 13.2394 - val_loss: 3.5487 - val_mse: 3.5487 - val_root_mean_squared_error: 1.8838 - val_mean_absolute_percentage_error: 15.4358
Epoch 284/300
71/71 [==============================] - 0s 1ms/step - loss: 2.3135 - mse: 2.3135 - root_mean_squared_error: 1.5210 - mea

In [30]:
r2_score(y_train, model_1.predict(Z_train))

0.9355177454230842

In [31]:
r2_score(y_test, model_1.predict(Z_test))

0.6582952411373528

In [43]:
nn_train = model_1.predict(Z_train)
nn_test = model_1.predict(Z_test)
nn_train = [g[0] for g in nn_train]
nn_test = [h[0] for h in nn_test]

### Second level model based on linear regression and neural network preds

In [34]:
a =pd.DataFrame([lin_train, nn_train, y_train], index=['lin_reg', 'nn', 'y']).T
b = pd.DataFrame([lin_test, nn_test, y_test], index=['lin_reg', 'nn', 'y']).T

In [35]:
a.head()

,lin_reg,nn,y
0,10.922396,10.147157,11.7
1,7.817903,8.496628,9.9
2,12.166780,8.927699,8.0
3,12.499178,11.561239,11.6
4,9.030794,7.003753,7.4


In [36]:
b.head()

,lin_reg,nn,y
0,12.136029,11.443439,12.5
1,10.070358,9.085341,5.9
2,11.389406,10.169192,10.7
3,7.893248,6.928510,6.3
4,12.921743,11.743506,12.4


In [37]:
Xa = a.drop(columns='y')
ya = a['y']
Xa_train, Xa_test, ya_train, ya_test = train_test_split(Xa, ya, random_state=42)

In [38]:
lev2 = LinearRegression()

In [39]:
lev2.fit(Xa_train, ya_train)

LinearRegression()

In [40]:
lev2.score(Xa_train, ya_train), lev2.score(Xa_test, ya_test), cross_val_score(lev2, Xa_train, ya_train).mean()

(0.9360029371402225, 0.9378826999899946, 0.9351672423186898)

These scores are scarring us

In [41]:
#Unseen data test data from lvl 1
lev2.score(b.drop(columns='y'), b['y'])

0.6561046020612404

In [42]:
#(0.9009880466719563, 0.6166400989225644, 0.8996610030098859)
#(0.8909971442563475, 0.5768477536960713, 0.888529811837276)
#(0.8841509454257197, 0.5772424027671583, 0.8834176035984036)

#(0.9118854053242108, 0.9288400370503621, 0.9114631988363131)